# Récuprération des données

On retrouve ici les fonctions nécéssaires pour récupérer un base de données propre avec les données qui nous intéresse.
La première chose à faire est de connaître les joueurs qui nous intéresse car les appels API sont limités.

In [1]:
import time
import datetime

%run Exploration.ipynb

## 1 - Séléction des joueurs actifs

Pour pouvoir appeler l'API uniquement sur les joueurs actifs il faut savoir sur quelle années chaque joueur a joué.
On est parti du principe qu'un joueur n'a joué pendant une saison que s'il a des statistiques sur cette saison.

In [2]:
nba_players = pd.DataFrame(players.get_players()).set_index('id')
nba_teams = pd.DataFrame(teams.get_teams()).set_index('id')

In [3]:
def get_data_players():
    '''Récupère les statistiques par années de chaque joueur
    
    Écrit dans le dossier data un fichier contenant toutes les statistiques
    annuelles des joueurs.
    '''
    
    df = pd.DataFrame()

    lst_index = nba_players.index.values
    
    wait_time = len(lst_index)*0.7

    print(f'Cela va prendre environ : {datetime.timedelta(seconds=wait_time)}')
    for i in range(len(lst_index)):
        print(f'\r{i+1}/{len(lst_index)}', end='')
        df = df.append(nba.PlayerCareerStats(lst_index[i])\
                            .season_totals_regular_season.get_data_frame())
        time.sleep(0.5)
    
    df.to_csv('../data/players_data.csv', index=False)

In [4]:
#get_data_players()

df_players = pd.read_csv('../data/players_data.csv')
df_players.sample(3)

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
5914,76539,1986-87,0,1610612751,NJN,30.0,6,2.0,106.0,20,32,0.625,0.0,0.0,0.000,17,24,0.708,9.0,10.0,19.0,2,2.0,3.0,15.0,25,57
8854,101162,2008-09,0,1610612753,ORL,25.0,63,3.0,794.0,106,187,0.567,1.0,1.0,1.000,26,45,0.578,96.0,190.0,286.0,14,18.0,53.0,25.0,111,239
25048,101124,2008-09,0,1610612763,MEM,26.0,82,7.0,2029.0,340,693,0.491,5.0,23.0,0.217,266,374,0.711,129.0,277.0,406.0,66,48.0,39.0,98.0,170,951


In [5]:
df_players['PLAYER_ID'].nunique() == len(nba_players)

False

On remarque que les listes des joueurs de l'API et du module ne sont pas les mêmes.

In [6]:
all_player = pd.DataFrame(nba_players.index)
missing = players_id2name(all_player[~all_player['id'] \
                                     .isin(df_players['PLAYER_ID'])], 'id')
missing.sample(3)

,id,NAME
4064,201987,Robert Vaden
579,202364,Da'Sean Butler
4403,1629685,Dylan Windler


Après une recherche sur internet de certains joueurs on se rends compte que ce sont des joueurs qui n'ont joué aucun match en NBA de leur carrière donc on peut ne pas en tenir compte. À partir de maintenenant on ne concidère que les joueurs de l'API et non du module.

In [7]:
def all_season_data():
    '''Récupère les saisons pour lesquels les joueurs ont joué'''
    
    df_teams = pd.read_csv('../data/players_data.csv')
    
    #On enlève les lignes TOT qui contabilise les statistiques 
    #lorsqu'un joueur a jouer dans plusieurs équipe une même année
    df_teams = df_teams[~(df_teams['TEAM_ABBREVIATION'] == 'TOT')]
    
    df = pd.get_dummies(df_teams[['PLAYER_ID', 'SEASON_ID']], 
                        columns=['SEASON_ID'], prefix='', prefix_sep='')
    
    df_active=df.groupby('PLAYER_ID').sum()
    
    return(df_active.reset_index())

In [8]:
# Cette 'matrice' est ce que l'on appelle 'sprase' ou creuse donc pour la garder
# en mémoire on peut utiliser un type d'encodage mais ici on ne s'embète pas.
all_season_data().sample(3)

,PLAYER_ID,1946-47,1947-48,1948-49,1949-50,1950-51,1951-52,1952-53,1953-54,1954-55,1955-56,1956-57,1957-58,1958-59,1959-60,1960-61,1961-62,1962-63,1963-64,1964-65,1965-66,1966-67,1967-68,1968-69,1969-70,1970-71,1971-72,1972-73,1973-74,1974-75,1975-76,1976-77,1977-78,1978-79,1979-80,1980-81,1981-82,1982-83,1983-84,1984-85,1985-86,1986-87,1987-88,1988-89,1989-90,1990-91,1991-92,1992-93,1993-94,1994-95,1995-96,1996-97,1997-98,1998-99,1999-00,2000-01,2001-02,2002-03,2003-04,2004-05,2005-06,2006-07,2007-08,2008-09,2009-10,2010-11,2011-12,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20
505,1096,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3428,201144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1
3447,201163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,1,1,1,1,0,1,1,2,1


In [9]:
def active_players(saison):
    '''Liste des PLAYER_ID des joueurs qui ont joué durant la saison'''
    
    df_season = all_season_data()
    return(df_season[df_season[season_id(saison)] > 0].PLAYER_ID.values)

In [10]:
active_players(2019)[:3]

array([1713, 2037, 2199])

## 2 - Séléction et mise en forme des statistiques

Maintenant que l'on a les joueurs qui ont joué on va récupéré toutes les informations sur leurs stats. 

In [11]:
def get_data_box(saison):
    '''Récupère les boxscores de tous les matchs de tous les joueurs
    qui ont joué pendant la saison
    ''' 
    
    df = pd.DataFrame()
    id_actives = active_players(saison)

    wait_time = len(id_actives)*0.7
    
    print(f'Cela va prendre environ : {datetime.timedelta(seconds=wait_time)}')
    for i in range(len(id_actives)):
        print(f'\r{i+1}/{len(id_actives)}', end='')
        
        df = df.append(boxscore_saison(id_actives[i], saison))
        time.sleep(0.5)
        
    df.to_csv(f'../data/boxscore_{saison}.csv', index=False)

In [12]:
def score_df(stats):
    '''Mets en forme et calcul le score de la base de donnée que l'on récupère 
    par le endpoint PlayerGameLog
    '''
    
    stats = stats.assign(GAME_DATE=pd.to_datetime(stats['GAME_DATE']),
                         TEAM=lambda df: df['MATCHUP'].str[:3],
                         ADV=lambda df: df['MATCHUP'].str[-3:],
                         HOME=lambda df: ~df['MATCHUP'].str.contains('@'),
                         WIN=lambda df: df['WL'].map({'W':True, 'L':False}),
                         FGF=lambda df: df['FGA'] - df['FGM'],
                         FG3F=lambda df: df['FG3A'] - df['FG3M'],
                         FTF=lambda df: df['FTA'] - df['FTM'])
    
    bonus = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M']
    malus = ['TOV', 'FGF', 'FG3F', 'FTF']

    cols = stats.columns.tolist()
    stats = stats[cols[:4] + cols[-7:-3] + bonus + malus]
    
    stats = stats.assign(BONUS=lambda df: df[bonus].sum(axis=1).astype('int'),
                         MALUS=lambda df: df[malus].sum(axis=1).astype('int'),
                         SCORE=lambda df: df['BONUS'] - df['MALUS'])
    
    #Cohérence des noms de colones
    stats.rename(columns={'Player_ID': 'PLAYER_ID', 'Game_ID': 'GAME_ID'}, 
                 inplace=True)
    
    #On mets l'ordre par rapport au joueur puis par dates
    stats.sort_values(['PLAYER_ID', 'GAME_DATE'], inplace=True)

    return(stats.reset_index(drop=True))

In [13]:
#get_data_box(2019)

score_df(pd.read_csv('../data/boxscore_2019.csv')).head(3)

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE,TEAM,ADV,HOME,WIN,PTS,REB,AST,STL,BLK,FGM,FTM,FG3M,TOV,FGF,FG3F,FTF,BONUS,MALUS,SCORE
0,22019,1713,21900014,2019-10-24,ATL,DET,False,True,0,0,1,0,0,0,0,0,1,2,2,0,1,5,-4
1,22019,1713,21900028,2019-10-26,ATL,ORL,True,True,2,2,1,1,0,1,0,0,0,2,2,0,7,4,3
2,22019,1713,21900043,2019-10-28,ATL,PHI,True,False,4,3,1,1,0,2,0,0,0,3,3,0,11,6,5
